# MREN223 Lab1 - Pedometer

In this lab, we will look at frequency spectrums, Fourier analysis, and signal filtering to design a pedometer. 

A pedometer counts each step a person takes by detecting the motion of the person's hands or hips. We will use the inertial measurement unit (IMU), containing a 3-axis accelerometer, on an Arduino Nano to generate motion data of a hand, and then process this data to count the number of steps.

In this lab, we will be using the following libraries:
`pip install scipy numpy matplotlib`

We advise you to scroll down to the lab description, and do steps (1) and (2) to get the Arduino work started. As it is taking its time to upload the sketch to the board, you can come up here and read the lab introduction. 

In [ ]:
import numpy as np
from scipy import signal
from scipy.fft import fft, ifft, fftfreq, fftshift
import matplotlib.pyplot as plt
%matplotlib inline

## Learning outcomes

1. Learn to compute the Fourier and inverse Fourier transforms on Python
2. Understand and generate frequency spectrums
3. Understand and generate frequency response
4. Learn filter types, how to design a filter and apply it to a signal
5. Apply FFT and filter design to design a pedometer

## Fast Fourier Transform

FFT is an algorithm which enables the calculation of the discrete Fourier Transform (DFT) efficiently.

We will start with an exercise to see what `fft` does. Recall that the fourier transform of a discrete signal $x[n]$ with period $N$ is
$$y[k] = \sum_{n=0}^{N-1} x[n] e^{-j \frac{2 \pi}{N} k}$$

*Exercise:* Consider the discrete signal $x[n]$ which involves a repeating pattern $[1,2,1,-1,1]$. Calculate its Fourier transform $y[k]$.

We will now compute $y[k]$ using `scipy.fft.fft`.

In [ ]:
x = np.array([1.0, 2.0, 1.0, -1.0, 1])

y = fft(x)
y

We can retreive $x[n]$ by computing the inverse Fourier transform using `scipy.fft.ifft`

In [ ]:
yinv = ifft(y)
yinv

###  Visualizing FFT: Frequency Spectrum

Let's now consider the sum of two complex exponential signals with frequencies $1$ and $5$ Hz.

$$y(t) = e^{j \omega_1 t} + e^{j \omega_2 t}$$
$$y[n] = y(nT_s)$$
$$\omega_1 = 2\pi(1), \omega_2 = 2\pi(5) \text{rad/s}$$

In [ ]:
# number of signal points and sample spacing
N, Ts = (200,  1./100.)

t = np.linspace(0.0, N*Ts, N, endpoint=False)
y = np.exp(2 * 1.j * 2.0*np.pi * t) + 0.5*np.exp(5 * 1.j * 2.0*np.pi * t)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

ax1.plot(t, np.real(y))
ax1.set_ylabel('$Real\{y[n]\}$')

ax2.plot(t, np.imag(y))
ax2.set_ylabel('$Imag\{y[n]\}$')

ax2.set_xlabel('Time (s)')
plt.tight_layout()

The FFT plot of a signal, called the **frequency spectrum** of the signal, displays the frequencies within that signal. For example, the frequency spectrum of the sum of complex exponential signals shows two peaks at the frequencies $1$ and $5$ Hz. 

The magnitude of the peaks informs of the coefficients preceding each exponential, showing that the $1$ Hz component is larger (or more dominant).

In [ ]:
yf = fft(y) # fft of the signal 

xf = fftfreq(N, Ts)
xf = fftshift(xf) #  preparing the x-axis for the spectrum

yplot = fftshift(yf)

plt.plot(xf, 1.0/N * np.abs(yplot))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.grid()

*Exercise:* Change the frequencies of the complex exponentials and see how they change the frequency spectrum. Test negative frequencies. Note: ensure the sampling frequency $f_s = 1/T_s$ is more than twice the largest frequency in the signals you test.

Similarly, we can do the same with sinusoidal signals. Consider a composite signal of three sinosoids with frequencies $3$ and $5$ and $40$ Hz and a ramp as given by

$$y(t) = cos(2\pi(3) t) + cos(2\pi(5) t) + cos(2\pi(40) t) + 0.1t$$
$$y[n] = y(nTs)$$

In [ ]:
# Number of sample points
N, Ts = (300, 1/100)

t = np.linspace(0.0, N*Ts, N, endpoint=False)

y = np.cos(3.0 * 2.0*np.pi*t) + 0.8*np.cos(5.0 * 2.0*np.pi*t) + 0.1*t + 0.1*np.cos(40.0 * 2.0*np.pi*t)

plt.plot(t, y)
plt.plot(t, 0.1*t)
plt.xlabel('Time (s)')
plt.ylabel('$y[n]$');

We find in the frequency spectrum peaks for each of these three frequencies and a peak at 0 rad/s for the ramp.

In [ ]:
yf = fft(y)
xf = fftfreq(N, Ts)
xf = fftshift(xf)
yplot = fftshift(yf)

plt.plot(xf, 1.0/N * np.abs(yplot))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.grid();

## Filtering

We will now design a filter to remove the frequencies we may not want in the signal.

Filters can be categorized into four categories:
1) low-pass: passes low frequencies and attenuates high frequencies
2) high-pass: passes high frequencies and attenuates low frequencies
3) band-pass: passes frequencies within a specified frequency range and attenuates all other frequencies
4) band-stop: attenutaes frequencies within a specified frequency range and passes all other frequencies

The `butterworth` filter in `scipy` provides a function which enables designing any of these filters with ease.
We will design a band-pass filter to filter out the ramp and the high frequency harmonic in the above composite signal.

First, we will visualize the filter.

In [ ]:
b, a = signal.butter(4, [1, 15], 'bandpass', analog=True)  #4 is filter order, the higher the more precise the filter, 
                                                            #[1,15] band of frequencies in rad/s to pass
w, h = signal.freqs(b, a) # computes the filter's frequency response

plt.semilogx(w, 20*np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency (rad/s)')
plt.ylabel('Magnitude (dB)')
plt.grid(which='both', axis='both')

plt.axvline(1, color='green') 
plt.axvline(15, color='green');

A filter's **frequency response** displays the action of the filter on different frequencies. 
The x-axis is the frequency in the input signal and the y-axis is the magnitude of amplification of that frequency in the input signal. 

The y-axis is logarithmic, in units of dB ($=20 \log_{10}(\text{Magnitude})$). A magnitude of 1 = 0 dB; hence, frequencies that 'see' a response of 0 dB are passed as they are. Frequencies that see a response higher (lower) that 0 dB are amplified (attentuated). 

The band of the bandpass filter is between the green vertical lines.

In [ ]:
sig = y # signal from before
sos = signal.butter(4, [1, 10], 'bandpass', fs=100, output='sos')
filtered = signal.sosfilt(sos, sig) # filtered signal

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

ax1.plot(t, sig)
ax1.set_title('Before filtering')
ax1.set_ylabel('y[n]')

ax2.plot(t, filtered)
ax2.set_title('After filtering')
ax1.set_ylabel('y[n]')

ax2.set_xlabel('Time (s)')
plt.tight_layout();

To confirm that the unwanted frequencies have been filtered out, we will plot the frequency spectrum of $y[n]$ following filtering.

In [ ]:
y = filtered
yf = fft(y)
xf = fftfreq(N, Ts)
xf = fftshift(xf)
yplot = fftshift(yf)

plt.plot(xf, 1.0/N * np.abs(yplot))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.grid();

## Lab Description

Retreive one Arduino Nano 33 BLE for your group. 

Arduino is available on Queen's lab computers on AppsAnywhere. Access https://apps.engineering.queensu.ca and open the Arduino IDE. You might have to open it twice, as it might fail to open the first time.

If you are new to Arduino, have a look at the Arduino provided tutorials:https://www.arduino.cc/en/Tutorial/HomePage
or https://technobyte.org/arduino-ide-complete-guide-beginners/

Once you have familiarized yourself with Arduino. We will do the following:

1) Follow the following tutorial (https://docs.arduino.cc/tutorials/nano-33-ble/imu-accelerometer), specific to Arduino Nano 33 BLE, to learn how to retrieve accelerometer data. 
The tutorial prints the accelerometer data to the Arduino IDE Serial window (https://www.arduino.cc/reference/en/language/functions/communication/serial/)

Make sure you have done all the following:
- Installed the board manager: Tools -> Boards Manager -> Arduino Mbed OS Nano Boards
- Installed the library: Tools -> Manage Libraries -> LSM9DS1
- Collected the accelerometer example code: File -> Examples -> Examples for any board -> Arduino_LSM9DS1 -> Simple Accelerometer
- Selected the board: Tools -> Board -> Arduino Mbed OS Nano Boards -> Arduino Nano 33 BLE; 
- Selected the port: Tools -> Port: COM# (Arduino Nano 33 BLE); 

2) Compile and upload the accelerometer example to the Arduino. This might take some time. 

3) Open the Serial monitor. Confirm that the accelerometer's x,y, and z values are printing onto the Serial monitor. 
You can clear the output on the Serial monitor.

4) Hold the Arduino Nano in your hand and simulate a few walking steps while collecting accelerometer data from the Arduino. 
Keep a count of your steps. 
Your lab partner(s) can inspect the Arduino Serial plotter as you walk to get an idea of how the data looks like for one walking step.
Select all (Ctrl-A) and copy (Ctrl-C) the accelerometer output from the Serial monitor.
Collect data for multiple (e.g., 5) runs, each with a different step count, to validate your method.

5) Import the accelerometer data into Python.
Have one array hold each of the accelerometer x, y and z axes output signal.

6) Inspect (visualize) each of the output signals from the accelerometer x, y and z axes. Select the ax(e)s that you can process to count the number of steps. Develop a strategy to count the steps from the signal(s).
You can resample the accelerometer data at a lower sampling rate (if you wish to).

7) Pre-process the data: trim the data; design a filter to remove any noise or unwanted drift from the selected signal(s); etc. 
- Plot the filter's frequency response.
- Plot a section of the selected signal(s) before and after filtering.
- Plot the frequency spectrum(s) of the selected signal(s) before and after filtering.

8) Write a python function that reads accelerometer data [float array] and outputs the number of steps taken [int]. 